# topic modeling playground
In this notebook, different concepts for topic modeling will be tested and evaluated.

In [252]:
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# preprocessing
import nltk
nltk.download('stopwords')
import spacy

# Plotting
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fabik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Structure

In [253]:
from enum import Enum


class Party(Enum):
    AFD = 0
    CDU = 1
    FDP = 2
    GRUENE = 3
    LINKE = 4
    SPD = 5



## Preprocessing

In [254]:
nlp = spacy.load('de_core_news_md')
from nltk.corpus import stopwords
from spacy.lang.de.stop_words import STOP_WORDS

# stopwords
nltk_stopwords = stopwords.words('german')

# build stopwords list
all_stopwords = list(set(STOP_WORDS) | set(nltk_stopwords))
with open('custom_stopwords.txt', 'r', encoding='utf-8') as f:
    all_stopwords += [line.strip() for line in f.readlines()]

# Load files
party_text = {}
for party in Party:
    all_stopwords.extend(['{}'.format(party.name.lower())])
    with open('resources/' + party.name + '.txt', encoding='utf-8', errors='ignore') as txt:
        file = " ".join(l for l in txt)
        # remove gender *
        file = re.sub(r'\*innen(\w*)\s', r'\1 ', file)
    party_text[party] = file



In [255]:

sections = re.split(r'\n\s*\n', party_text[Party.FDP])

partie_wordbags_mod = []


for section in sections:

    partie_wordbag = gensim.utils.simple_preprocess(section)

    #filter stopwords
    partie_wordbag_spacy = [word for word in partie_wordbag if word not in all_stopwords]

    partie_wordbags_mod.append(partie_wordbag_spacy)


# make bigrams
bigram = gensim.models.Phrases(partie_wordbags_mod, min_count=5, threshold=100)
# make trigrams
trigram = gensim.models.Phrases(bigram[partie_wordbags_mod], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags and token.lemma_ not in all_stopwords])
    return texts_out

words_trigrams = make_trigrams(partie_wordbags_mod)
words_lematized = lemmatization(words_trigrams)

id2word = corpora.Dictionary(words_lematized)
texts = words_lematized
corpus = [id2word.doc2bow(text) for text in texts]




In [256]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=10,
                                             )

lda_model.print_topics(num_topics=10, num_words=10)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words_lematized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.560801052906601

Coherence Score:  0.29001427202099356


##V

In [257]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

c:\users\fabik\pycharmprojects\textanalytics\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.040941  0.000342       1        1  22.340981
3     -0.026373 -0.042660       2        1  15.554121
7     -0.014604  0.034789       3        1  14.415790
2     -0.021328 -0.020526       4        1  12.682357
8      0.004593  0.012487       5        1  10.796629
6      0.059302 -0.026979       6        1   9.975493
9      0.016737  0.039101       7        1   6.953794
1     -0.015725  0.005334       8        1   3.222686
4      0.010589  0.000955       9        1   2.525996
0      0.027751 -0.002843      10        1   1.532153, topic_info=                Term       Freq      Total Category  logprob  loglift
341      unternehmen  56.000000  56.000000  Default  30.0000  30.0000
1400              co  18.000000  18.000000  Default  29.0000  29.0000
16           bildung  28.000000  28.000000  Default  28.0000  28.0000
94          schützen  19.000000  19.000000  Default  27.0000  27.0000
743   altersvorsorge  15.000000  15.000000  Default  26.0000  26.0000
...              ...        ...        ...      ...      ...      ...
320           privat   0.241224  24.190717  Topic10  -6.6183  -0.4295
129          zukunft   0.231111  23.638286  Topic10  -6.6611  -0.4492
354       wirtschaft   0.224467  20.101562  Topic10  -6.6903  -0.3163
760            daten   0.215698  16.150617  Topic10  -6.7301  -0.1373
16           bildung   0.217706  28.745834  Topic10  -6.7209  -0.7046

[744 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
613       1  0.593940  abgabenquote
613       3  0.296970  abgabenquote
2824      3  0.466747    abgeordnet
2824      5  0.466747    abgeordnet
875       1  0.582337    abgestimmt
...     ...       ...           ...
2578      3  0.168309   überwachung
2578      5  0.168309   überwachung
2578      7  0.168309   überwachung
255       3  0.352788   überzeugung
255       4  0.352788   überzeugung

[1412 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 8, 3, 9, 7, 10, 2, 5, 1])